In [2]:
import os
from typing import List
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_URL'] = os.getenv('OPENAI_BASE_URL')

# from langchain_community.chat_models.baidu_qianfan_endpoint import QianfanChatEndpoint
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, BaseMessage
from langgraph.graph import END, MessageGraph
import json
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool

if __name__ == '__main__':


    model = ChatOpenAI(model="glm-4-flash")


    @tool
    def multiply(first_number: int, second_number: int):
        """将2个数相乘"""
        return first_number * second_number


    model_with_tools = model.bind(tools=[convert_to_openai_tool(multiply)])

    # 定义一个消息图
    graph = MessageGraph()
    def invoke_model(state: List[BaseMessage]):
        return model_with_tools.invoke(state)

    # 定义一个节点
    graph.add_node("start", invoke_model)

    def invoke_tool(state: List[BaseMessage]):
        tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
        multiply_call = None

        for tool_call in tool_calls:
            if tool_call.get("function").get("name") == "multiply":
                multiply_call = tool_call

        if multiply_call is None:
            raise Exception("No adder input found.")

        res = multiply.invoke(
            json.loads(multiply_call.get("function").get("arguments"))
        )

        return ToolMessage(
            tool_call_id=multiply_call.get("id"),
            content=res
        )


    graph.add_node("multiply", invoke_tool)

    graph.add_edge("multiply", END)

    # 定义逻辑入口
    graph.set_entry_point("start")

    # 定义分支选择函数 router
    def router(state: List[BaseMessage]):
        tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
        if len(tool_calls):
            return "multiply"
        else:
            return "end"

    # 增加条件边
    graph.add_conditional_edges("start", router, {
        "multiply": "multiply",
        "end": END,
    })

    # 由图生成一个 runnable
    runnable = graph.compile()

    res = runnable.invoke(HumanMessage("8乘以4等于？"))
    pass
    res_2 = runnable.invoke(HumanMessage("你好！"))
    pass
    print(res)
    print(res_2)

[HumanMessage(content='8乘以4等于？', additional_kwargs={}, response_metadata={}, id='31d26493-7edc-407c-ba1c-82eb981d48cc'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_-8852873450729821057', 'function': {'arguments': '{"first_number": 8, "second_number": 4}', 'name': 'multiply'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 170, 'total_tokens': 204, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4-flash', 'system_fingerprint': None, 'id': '20250327111158f90cb2e95b734c16', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8e3952fc-dd0f-4ef3-8f5a-98afb9a72fc2-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 8, 'second_number': 4}, 'id': 'call_-8852873450729821057', 'type': 'tool_call'}], usage_metadata={'input_tokens': 170, 'output_tokens': 34, 'total_tokens': 204, 'input_token_details': {}, 'output_token_details': {}}), Too